# Convolutional Neural Network

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importing the libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator  #data augumentation k lye

In [3]:
tf.__version__

'2.18.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
!cp -r "/content/drive/MyDrive/cnn dataset/brain tumor" ./

In [5]:
# path hold kre hai local copy ka !
train_dir = '/content/brain tumor/Training/'  # Updated path
test_dir = '/content/brain tumor/Testing/'  # Updated path

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,  # Random 20* rotations
    width_shift_range=0.2,  # Horizontal shifting 20%
    height_shift_range=0.2,  # Vertical shifting
    brightness_range=[0.2,1.0]  # Random brightness
)


training_set = train_datagen.flow_from_directory(
    train_dir ,
    target_size=(128, 128),  # resize jo cnn mai feed ho
    batch_size=32,
    class_mode='categorical'  # 'categorical' for multi-class
)

Found 5712 images belonging to 4 classes.


### Preprocessing the Test set

In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
    test_dir ,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'  #  'categorical' for multi-class
)

Found 1311 images belonging to 4 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [8]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[128, 128, 3]))
cnn.add(tf.keras.layers.BatchNormalization())  #  Batch Normalization

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [10]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [11]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Adding a Third convolutional layer


In [12]:
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [13]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [14]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))  # Dropout layer to avoid overfitting

### Step 5 - Output Layer

In [15]:
cnn.add(tf.keras.layers.Dense(units=4, activation='softmax'))  # Softmax for multi-class classification

## Part 3 - Training the CNN

### Compiling the CNN

In [16]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [17]:
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 53s 254ms/step - accuracy: 0.3634 - loss: 3.7729 - val_accuracy: 0.2288 - val_loss: 25.9445
Epoch 2/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 70s 213ms/step - accuracy: 0.4244 - loss: 1.2279 - val_accuracy: 0.4142 - val_loss: 1.6861
Epoch 3/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 37s 208ms/step - accuracy: 0.4618 - loss: 1.1808 - val_accuracy: 0.4958 - val_loss: 1.0592
Epoch 4/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 38s 211ms/step - accuracy: 0.5022 - loss: 1.1105 - val_accuracy: 0.4638 - val_loss: 1.4313
Epoch 5/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 39s 217ms/step - accuracy: 0.5166 - loss: 1.0860 - val_accuracy: 0.3593 - val_loss: 4.1296
Epoch 6/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 39s 218ms/step - accuracy: 0.5306 - loss: 1.0375 - val_accuracy: 0.3753 - val_loss: 5.2735
Epoch 7/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 38s 212ms/step - accuracy: 0.5549 - loss: 1.0120 - val_accuracy: 0.3753 - val_loss: 3.0423
Epoch 8/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 38s 211ms/step - accuracy: 0.5750 - loss: 

In [18]:
# Saving Trainned Model to Google Drive
from google.colab import drive
drive.mount('/content/drive')

cnn.save('/content/drive/MyDrive/brain_tumor_model.h5')  # Save to Google Drive


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


LOAD MODEL FROM HERE


In [19]:
from tensorflow.keras.models import load_model

loaded_model = load_model('/content/drive/MyDrive/brain_tumor_model.h5')
loaded_model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,306,054 (12.61 MB)

 Trainable params: 3,305,604 (12.61 MB)

 Non-trainable params: 448 (1.75 KB)

 Optimizer params: 2 (12.00 B)

## Part 4 - Making a  prediction

In [20]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the test image size bhi adjust keya
test_image = image.load_img('/content/brain tumor/Predictions/p.jpg', target_size=(128, 128))

# image -----> numpy array
test_image = image.img_to_array(test_image)


test_image = np.expand_dims(test_image, axis=0) #batch size match k lye dimension bhda rhe h


test_image = test_image / 255.0

# Make prediction
predicted_class = cnn.predict(test_image)


class_labels = ['glioma', 'meningioma', 'notumor', 'pituitary']


# Find the index of the predicted class
predicted_class_index = np.argmax(predicted_class, axis=1)[0]

# Get the predicted class label
predicted_label = class_labels[predicted_class_index]

print(f"The predicted class is: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 684ms/step
The predicted class is: pituitary
